# Autoencoders

[sklearn MLP](https://qu4nt.github.io/sklearn-doc-es/modules/generated/sklearn.neural_network.MLPClassifier.html)

[Visualizacion](https://github.com/jzliu-100/visualize-neural-network/tree/master)

[Explicacion de Patrick Winston](https://ocw.mit.edu/courses/6-034-artificial-intelligence-fall-2010/resources/lecture-12b-deep-neural-nets/)



# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/deeplearning/deeplearning.html)